<h3>1. 필요한 패기지 INSTALL

In [1]:
!pip install selenium beautifulsoup4 requests webdriver-manager


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


<h3>2. 패키지 IMPORT

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import requests
import os

<h3>3. URL 설정 함수

In [3]:
# URL을 생성하는 함수
def get_url(gender='', season='', category=''):
    base_url = 'https://www.musinsa.com/snap/main/recommend'
    params = {
        'genders': gender,
        'seasons': season,
        'category': category
    }
    query_string = '&'.join([f"{key}={value}" for key, value in params.items() if value])
    return f"{base_url}?{query_string}"

<h3>4. 이미지 크롤링 함수

In [9]:
# 설정된 URL에서 이미지를 크롤링하고 다운로드하는 함수
def crawl_images(driver, gender, season, category):
    # 성별, 계절, 스타일, 카테고리 이름 매핑
    gender_map = {'MEN': '남', 'WOMEN': '여'}
    season_map = {'1': '봄', '3': '여름', '2': '가을', '4': '겨울'}
    style_map = {
        '8': '캐주얼', '10': '스트릿', '5': '미니멀', '14': '스포티', '9': '클래식',
        '16': '시크', '20': '고프코어', '7': '시티보이', '18': '레트로', '15': '리조트',
        '17': '프레피', '19': '에스닉', '12': '로맨틱', '6': '워크웨어', '13': '걸러시'
    }
    category_map = {'003002': '데님팬츠', '002022': '후드집업', '002007': '코트'}

    # 폴더 이름 생성
    gender_name = gender_map.get(gender, gender)
    season_name = season_map.get(season, season)
    category_name = category_map.get(category, category)

    # 스크롤 시뮬레이션
    scroll_pause_time = 5
    screen_height = driver.execute_script("return window.screen.height;")
    i = 1

    downloaded_count = 0
    max_downloads = 1000

    # 저장 디렉토리 생성 (C:\Users\jrnee\Desktop\무신사크롤링\ 성별/계절/스타일/카테고리 구조)
    base_dir = "/home/sejong/Desktop/junseok/SPRC/musinsa/snap"
    output_dir = os.path.join(base_dir, gender_name, category_name)
    os.makedirs(output_dir, exist_ok=True)

    while downloaded_count < max_downloads:
        driver.execute_script(f"window.scrollTo(0, {screen_height}*{i});")
        time.sleep(scroll_pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight;")
        if new_height == screen_height * i:
            break
        i += 1

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 이미지 URL 추출
        images = soup.select('img.max-w-full.w-full.absolute.m-auto.inset-0.h-auto.z-0.visible.object-cover')
        image_urls = [img['src'] for img in images]

        # 이미지 다운로드
        for img_url in image_urls:
            if downloaded_count >= max_downloads:  # 전체 다운로드 수 제한
                print(f"Downloaded {downloaded_count} images. Stopping further downloads.")
                return
            img_name = os.path.basename(img_url.split('?')[0])  # URL에서 쿼리 스트링 제거
            # 동일 이름의 파일이 이미 존재하는지 확인
            img_path = os.path.join(output_dir, f"{img_name}.jpg")
            if os.path.exists(img_path):
                print(f"이미 동일 이름의 파일이 존재하여 스킵: {img_name}.jpg")
                continue
            img_response = requests.get(img_url)
            with open(os.path.join(output_dir, img_name), 'wb') as f:
                f.write(img_response.content)
            downloaded_count += 1

    print(f"Downloaded {downloaded_count} images to {output_dir}")

<h3>5. 크롤링 실행

In [10]:
# 크롤링 실행
def main():
    genders = ['MEN', 'WOMEN']  # 성별 리스트
    season = '4'  # 계절 고정 (4: 겨울)
    styles = ['8', '10', '5', '14', '9', '16', '20', '17', '19', '15', '18', '7', '12', '6', '13']  # 스타일 리스트
    categories = ['002007'] # 카테고리 (003002: 데님팬츠츠)'002022', '003002', 

    # 웹 드라이버 설정
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    try:
        for gender in genders:
            for category in categories:
                url = get_url(gender=gender, season=season, category=category)
                print(f"Generated URL: {url}")
                driver.get(url)
                crawl_images(driver, gender, season, category)
    finally:
        driver.quit()

if __name__ == "__main__":
    main()

Generated URL: https://www.musinsa.com/snap/main/recommend?genders=MEN&seasons=4&category=002007
Downloaded 1000 images. Stopping further downloads.
Generated URL: https://www.musinsa.com/snap/main/recommend?genders=WOMEN&seasons=4&category=002007
Downloaded 1000 images. Stopping further downloads.


In [11]:
def count_files_in_folder(folder_path):
    """
    폴더 내 파일 개수를 확인하는 함수.

    Parameters:
    - folder_path (str): 파일 개수를 확인할 폴더 경로.
    - include_subfolders (bool): 하위 디렉토리 파일도 포함할지 여부 (기본값: False).

    Returns:
    - int: 폴더 내 파일 개수.
    """
    if not os.path.exists(folder_path):
        print(f"지정된 경로가 존재하지 않습니다: {folder_path}")
        return 0
    
    file_count = 0

    file_count = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])

    return file_count

# 사용 예시
for gender in ['남', '여']:
    for category in ['데님팬츠', '코트', '후드집업']:

        folder_path = f"/home/sejong/Desktop/junseok/SPRC/musinsa/snap/{gender}/{category}"
        print(f"{gender}/{category} 폴더 내 파일 개수:", count_files_in_folder(folder_path))

남/데님팬츠 폴더 내 파일 개수: 410
남/코트 폴더 내 파일 개수: 172
남/후드집업 폴더 내 파일 개수: 410
여/데님팬츠 폴더 내 파일 개수: 410
여/코트 폴더 내 파일 개수: 98
여/후드집업 폴더 내 파일 개수: 413


<h2>Done